Alissa Crist
BU MET CS 689, Spring O2 2024
4/11/2024
Term Project Update 3, ETL

Import libraries, set options

In [1]:
import pandas as pd
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

Load .csv files into dataframes

In [2]:
# Load Permit Appliactions file into df
permit_df = pd.read_csv("Files/Permit_Applications_20240410.csv", low_memory=False)
# Load Planning Appliactions file into df
planning_df = pd.read_csv("Files/Planning_Applications_20240410.csv", low_memory=False)
# Load Orlando Workforce file into df
workforce_df = pd.read_csv("Files/Orlando_Workforce.csv", low_memory=False)
# Load Orlando Housing Data file into df
housing_df = pd.read_csv("Files/Orlando_Housing_Data.csv", low_memory=False)

Adjust date formatting on permit_df

In [3]:
permit_df['Processed Date'] = pd.to_datetime(permit_df['Processed Date'], format='%m/%d/%Y')

Select only permits newer than Jan 1, 2022

In [4]:
permit_df_2022 = permit_df.loc[(permit_df['Processed Date'] >= '01/01/2022')]

Adjust date formatting on planning_df

In [5]:
planning_df['Application Date'] = pd.to_datetime(planning_df['Application Date'], format='%Y-%m-%d')
planning_df['Approval Date'] = pd.to_datetime(planning_df['Approval Date'], format='%Y-%m-%d')
planning_df['Last Action Date'] = pd.to_datetime(planning_df['Last Action Date'], format='%Y-%m-%d')

Select only planning permits newer than Jan 1, 2022

In [6]:
planning_df_2022 = planning_df.loc[(planning_df['Application Date'] >= '01/01/2022')]

Create subset of permit data with just the attributes we need

In [7]:
permit_df_2022_sub = permit_df_2022[['Permit Number', 'Application Type', 'Plan Review Type', 'Project Name', 'Permit Address',
                                     'Parcel Number', 'Parcel Owner Name', 'Property Owner Name', 'Square Footage', 'Neighborhood',
                                     'Contractor Name', 'Contractor', 'Contractor Address', 'Commissioner District', 'Mainstreet District',
                                     'Estimated Cost', 'Application Status', 'Processed Date', 'Issue Permit Date']]

One more date adjustment

In [8]:
permit_df_2022_sub['Issue Permit Date'] = pd.to_datetime(permit_df_2022_sub['Issue Permit Date'], format='%m/%d/%Y')
permit_df_2022_sub.head()

/var/folders/s_/qq24xl7d1s7f49945mqkqtx00000gn/T/ipykernel_12114/480771700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permit_df_2022_sub['Issue Permit Date'] = pd.to_datetime(permit_df_2022_sub['Issue Permit Date'], format='%m/%d/%Y')


,Permit Number,Application Type,Plan Review Type,Project Name,Permit Address,Parcel Number,Parcel Owner Name,Property Owner Name,Square Footage,Neighborhood,Contractor Name,Contractor,Contractor Address,Commissioner District,Mainstreet District,Estimated Cost,Application Status,Processed Date,Issue Permit Date
88,ENG2023-12665,Engineering,Residential 1/2,ADDITION,12062 AMPERSAND DR,312403897701590,ONEY GEORGE ALAN ONEY DEBRA ANN,"ONEY DEBRA ANN, ONEY GEORGE ALAN",0.0,Storey Park,NaN,NaN,NaN,1,NaN,0.0,Finaled,2023-11-07,2023-11-08
567,ENG2023-11739,Engineering,Residential 1/2,REPAIR AND REPLACE,7234 BLACK BULL LN,282302345400810,HUNSBERGER SHANE MICHAEL FRANCHIM SAMANTA CHI...,FRANCHIM SAMANTA CHIARINI,0.0,Metro West,ALPHA FOUNDATION SPECIALISTS INC,MATTHEW S SHANLEY,"P O BOX 14989,TALLAHASSEE, FL 32317",6,NaN,0.0,Finaled,2023-07-05,2023-07-10
659,SIF2021-186-14,School Impact Fees,Residential 1/2,Laureate Park Parcel N-3 Phase 2 107/135 Lot 80,10340 MARCONI AVE,312431484600800,CENTURY HOMES FLORIDA LLC,NaN,3062.0,Lake Nona South,NaN,NaN,NaN,1,NaN,0.0,Closed,2022-07-06,NaT
736,BLD2022-19978,Building Permit,Residential 1/2,HO CRU 21244,8600 HATA ST,312419780201140,CRUZ MIRLA M,CRUZ MIRLA M,0.0,Lake Nona South,HGA CONSTRUCTION,BRIAN LEAHY (HGA CONSTRUCTION INC),"1465 GENE STREET,WINTER PARK, FL 32789",1,NaN,15000.0,Open,2022-09-16,2023-01-04
855,BLD2023-10220,Building Permit,Commercial,Iron Bridge Fire Alarm Improvements,601 Iron Bridge Cir,312130317400021,NaN,No Contact,0.0,NaN,MILLENNIUM FIRE & SECURITY INC,CHRISTOPHER L MARTIN,"6160 EDGEWATER DR,ORLANDO, FL 32810",,NaN,120000.0,Open,2023-01-09,2023-02-28


Check unique values of Application Status - we need this later to split current and previous permits

In [9]:
permit_df_2022_sub['Plan Review Type'].unique()

array(['Residential 1/2', 'Commercial', 'Residential 3 or more',
       'No Plan Review Type'], dtype=object)

Create subset of planning data with just the attributes we need

In [10]:
planning_df_2022_sub = planning_df_2022[['Application Number', 'Application Name', 'Applicant', 'Address', 
                                         'Parcel ID', 'Commissioner District', 'Mainstreet District', 'Application Description', 
                                         'Neighborhood', 'Application Status', 'Application Date', 'Approval Date', 'Last Action Date']]

Rename Parcel column

In [11]:
planning_df_2022_sub.rename(columns={"Parcel ID":"Parcel Name"}, inplace= True)
   
planning_df_2022_sub.head()

/var/folders/s_/qq24xl7d1s7f49945mqkqtx00000gn/T/ipykernel_12114/1653012612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planning_df_2022_sub.rename(columns={"Parcel ID":"Parcel Name"}, inplace= True)


,Application Number,Application Name,Applicant,Address,Parcel Name,Commissioner District,Mainstreet District,Application Description,Neighborhood,Application Status,Application Date,Approval Date,Last Action Date
0,HPB2022-10099,54 N Orange Ave. (SWC,GEORGE F MALTEZOS,54 N ORANGE AVE,29-22-26-7352-36-011,5,City District,Historic Preservation Board,Central Business District,Closed,2022-04-05,2022-05-16,2024-01-15
1,HPB2022-10083,515 Delaney Ave. (Alterations),Elizabeth Marsh,515 DELANEY AVE,29-22-36-1988-02-030,4,NaN,Historic Preservation Board,Lake Cherokee,Closed,2022-03-22,2022-04-04,2024-01-15
4,HPB2022-10231,1104 E Livingston St.,Steve Voeller,1104 E LIVINGSTON ST,29-22-25-2508-05-080,4,NaN,Historic Preservation Board,Lake Eola Heights,Closed,2022-09-09,2022-12-01,2024-01-29
5,HPB2022-10241,137 E Concord St.,Confidential,137 E CONCORD ST,29-22-25-3704-00-101,4,NaN,Historic Preservation Board,Lake Eola Heights,Closed,2022-09-16,2022-09-19,2024-01-15
7,MOD2022-10030,1203 Reading Dr. - detached garage,ROBERT B DUKES,1203 READING DR,29-22-22-2412-10-101,3,NaN,Modification of Standards,College Park,Closed,2022-04-25,2022-08-17,2024-01-15


Upload the finished subsets to SQL

In [12]:
# Add an update timestamp column to permit and planning df
permit_df_2022_sub['Update Timestamp'] = datetime.now()
planning_df_2022_sub['Update Timestamp'] = datetime.now()

/var/folders/s_/qq24xl7d1s7f49945mqkqtx00000gn/T/ipykernel_12114/3962034922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permit_df_2022_sub['Update Timestamp'] = datetime.now()
/var/folders/s_/qq24xl7d1s7f49945mqkqtx00000gn/T/ipykernel_12114/3962034922.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planning_df_2022_sub['Update Timestamp'] = datetime.now()


In [13]:
from sqlalchemy import create_engine

# PostgreSQL connection details
dbname = 'acrist_cs689_termproj'
user = 'postgres'
password = 'postgres1092'
host = 'localhost' 

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

# Create staging tables in the orlando_development schema
permit_df_2022_sub.to_sql('Permit_Staging', engine, schema='orlando_development', if_exists='replace', index=False)
planning_df_2022_sub.to_sql('Planning_Staging', engine, schema='orlando_development', if_exists='replace', index=False)
workforce_df.to_sql('Workforce_Staging', engine, schema='orlando_development', if_exists='replace', index=False)
housing_df.to_sql('Housing_Staging', engine, schema='orlando_development', if_exists='replace', index=False)

# Commit changes
engine.dispose()